In [1]:
 !pip install transformers

In [2]:
import os
import pandas as pd
import numpy as np

In [3]:
pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.2/242.2 KB 1.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/6-class-bert-preprocessed-dataset/Bert_Preprocessed_Dataset.xlsx


In [5]:
df=pd.read_excel('/kaggle/input/6-class-bert-preprocessed-dataset/Bert_Preprocessed_Dataset.xlsx')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6405 entries, 0 to 6404
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  6405 non-null   int64 
 1   ID          6404 non-null   object
 2   Text        6404 non-null   object
 3   Sentiment   6405 non-null   object
 4   Label       6405 non-null   int64 
 5   Token       6405 non-null   object
 6   Token_ID    6405 non-null   object
dtypes: int64(2), object(5)
memory usage: 350.4+ KB


In [6]:
pip install ktrain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 21.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 KB 55.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.7/263.7 KB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 68.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 KB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 31.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) 

In [7]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 

In [8]:
import ktrain
from ktrain import text

In [9]:
train=df[:int(.8*len(df))]
test=df[int(.8*len(df)):int(.9*len(df))]
validation=df[int(.9*len(df)):]
x_train=train['Text'].tolist()
y_train=train['Label'].tolist()
x_validation=validation['Text'].tolist()
y_validation=validation['Label'].tolist()
x_test=test['Text'].tolist()
y_test=test['Sentiment'].tolist()

In [10]:
x_train= np.array(x_train)
x_validation= np.array(x_validation)
x_test= np.array(x_test)


In [11]:
categories = ['angry','disgust','fear','happy','sad','surpise']

In [12]:
model_name = 'monsoon-nlp/bangla-electra'
trans = text.Transformer(model_name,maxlen=50,class_names=categories)

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

In [13]:
train = trans.preprocess_train(x_train,y_train)
valid = trans.preprocess_test(x_validation,y_validation)

preprocessing train...
language: bn
train sequence lengths:
	mean : 16
	95percentile : 44
	99percentile : 102


Downloading:   0%|          | 0.00/161 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/492k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: bn
test sequence lengths:
	mean : 16
	95percentile : 49
	99percentile : 96


In [14]:
model = trans.get_classifier()

Downloading:   0%|          | 0.00/54.1M [00:00<?, ?B/s]

In [15]:
learner = ktrain.get_learner(model, train_data=train,val_data=valid, batch_size=12)

In [16]:
from sklearn.utils import class_weight
class_weight=class_weight.compute_class_weight(class_weight='balanced',classes=np.unique(y_train),y=y_train)
weight={i:class_weight[i] for i in range(6)}

In [17]:
learner.fit_onecycle(2e-5,20,class_weight=weight) 



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/20
427/427 [==============================] - 46s 61ms/step - loss: 1.7918 - accuracy: 0.1608 - val_loss: 1.7935 - val_accuracy: 0.2059
Epoch 2/20
427/427 [==============================] - 23s 53ms/step - loss: 1.7909 - accuracy: 0.1844 - val_loss: 1.7915 - val_accuracy: 0.2090
Epoch 3/20
427/427 [==============================] - 23s 53ms/step - loss: 1.7883 - accuracy: 0.1889 - val_loss: 1.7868 - val_accuracy: 0.2278
Epoch 4/20
427/427 [==============================] - 23s 53ms/step - loss: 1.7813 - accuracy: 0.2607 - val_loss: 1.7650 - val_accuracy: 0.3136
Epoch 5/20
427/427 [==============================] - 23s 54ms/step - loss: 1.7644 - accuracy: 0.3193 - val_loss: 1.7514 - val_accuracy: 0.3105
Epoch 6/20
427/427 [==============================] - 23s 54ms/step - loss: 1.7287 - accuracy: 0.3614 - val_loss: 1.6554 - val_accuracy: 0.3775
Epoch 7/20
427/427 [==============================] - 22s 52ms/step - los

In [18]:
learner.validate(class_names=categories)

              precision    recall  f1-score   support

       angry       0.51      0.58      0.54       137
     disgust       0.19      0.42      0.26        72
        fear       0.09      0.43      0.15        30
       happy       0.74      0.57      0.64       195
         sad       0.20      0.02      0.03       132
     surpise       0.37      0.09      0.15        75

    accuracy                           0.38       641
   macro avg       0.35      0.35      0.30       641
weighted avg       0.44      0.38      0.37       641



array([[ 80,  37,  13,   3,   0,   4],
       [ 22,  30,  13,   6,   1,   0],
       [  1,  13,  13,   3,   0,   0],
       [ 16,  14,  45, 111,   6,   3],
       [ 28,  36,  42,  19,   2,   5],
       [ 11,  28,  19,   9,   1,   7]])

In [19]:
predictor = ktrain.get_predictor(learner.model, preproc=trans)

In [20]:
y_pred = predictor.predict(x_test)
print(y_pred)

['angry', 'disgust', 'angry', 'angry', 'disgust', 'fear', 'fear', 'angry', 'happy', 'happy', 'disgust', 'angry', 'happy', 'fear', 'disgust', 'angry', 'happy', 'fear', 'surpise', 'happy', 'angry', 'angry', 'angry', 'fear', 'fear', 'happy', 'happy', 'angry', 'angry', 'fear', 'angry', 'angry', 'fear', 'happy', 'angry', 'angry', 'happy', 'angry', 'happy', 'happy', 'angry', 'surpise', 'fear', 'happy', 'angry', 'angry', 'angry', 'fear', 'disgust', 'angry', 'disgust', 'angry', 'angry', 'fear', 'angry', 'disgust', 'fear', 'disgust', 'disgust', 'angry', 'fear', 'fear', 'fear', 'happy', 'fear', 'fear', 'disgust', 'fear', 'fear', 'fear', 'happy', 'disgust', 'disgust', 'happy', 'disgust', 'fear', 'disgust', 'angry', 'fear', 'fear', 'disgust', 'angry', 'angry', 'angry', 'disgust', 'happy', 'happy', 'happy', 'disgust', 'angry', 'disgust', 'angry', 'happy', 'happy', 'disgust', 'fear', 'fear', 'angry', 'surpise', 'angry', 'fear', 'angry', 'disgust', 'angry', 'happy', 'fear', 'happy', 'happy', 'disgust

In [21]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       angry       0.52      0.58      0.55       144
     disgust       0.16      0.33      0.22        66
        fear       0.10      0.50      0.17        32
       happy       0.73      0.62      0.67       187
         sad       0.00      0.00      0.00       148
     surpise       0.12      0.03      0.05        63

    accuracy                           0.38       640
   macro avg       0.27      0.34      0.28       640
weighted avg       0.36      0.38      0.35       640

